# Wine Quality

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import ElasticNet
import mlflow
import mlflow.sklearn

In [ ]:
#mlflow.set_tracking_uri("sqlite:///wine-quality.sqlite")

In [ ]:
mlflow.get_tracking_uri()

In [ ]:
# Loading data from a CSV file
df_wine = pd.read_csv('./wine-quality.csv')

# Separating the target class ('quality') from remainder of the training data
X = df_wine.drop(columns = 'quality')

y = df_wine[['quality']]

# Splitting the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 42)

## Model Training and Logging

In [ ]:
# Fix Conda Python version used by Seldon Core MLflow in Kubernetes
conda_env = mlflow.sklearn.get_default_conda_env()

for i, dep in enumerate(conda_env['dependencies']):
    if type(dep) == str and dep.startswith("python="):
        conda_env['dependencies'][i] = "python"

conda_env

In [ ]:
# Defining model parameters
for name, alpha, l1_ratio in [("wine-model-a", 0.5, 0.5), ("wine-model-b", 1.0, 0.5)]:
    # Running MLFlow script
    with mlflow.start_run():

        # Instantiating model with model parameters
        model = ElasticNet(alpha = alpha, l1_ratio = l1_ratio)

        # Fitting training data to the model
        model.fit(X_train, y_train)

        # Running prediction on validation dataset
        preds = model.predict(X_val)

        # Getting metrics on the validation dataset
        rmse = mean_squared_error(preds, y_val)
        abs_error = mean_absolute_error(preds, y_val)
        r2 = r2_score(preds, y_val)

        # Logging params and metrics to MLFlow
        mlflow.log_param('alpha', alpha)
        mlflow.log_param('l1_ratio', l1_ratio)
        mlflow.log_metric('rmse', rmse)
        mlflow.log_metric('abs_error', abs_error)
        mlflow.log_metric('r2', r2)

        # Logging model to MLFlow. Requires MLflow (Server) with DB backend
        mlflow.sklearn.log_model(
            sk_model = model,
            artifact_path = "",
            registered_model_name = name,
            conda_env = conda_env
        )

In [ ]:
df = mlflow.search_runs(filter_string="metrics.rmse < 1")

In [ ]:
df

In [ ]:
#Fetching Run ID for best model
# run_id = df.loc[df['metrics.rmse'].idxmin()]['run_id']
#best_model_url = df.loc[df['metrics.rmse'].idxmin()]["artifact_uri"]
# Load model as a PyFuncModel.
#best_model = mlflow.pyfunc.load_model(best_model_url)
#y_pred = best_model.predict(X_val)
#print(y_pred[0:10])

## Deploy Models from MLflow to Seldon Core

In [ ]:
#from mlflow import pyfunc.load_model(model_uri="models:/wine-model-a")
from mlflow.tracking import MlflowClient

client = MlflowClient()

model_a_version = client.get_latest_versions("wine-model-a", stages=["None"])[0].version
model_b_version = client.get_latest_versions("wine-model-b", stages=["None"])[0].version

In [ ]:
import seldon_core_utils

ab = seldon_core_utils.ab_deployment(
    name="ab",
    namespace="default",
    secret_name="mlflow-seldon-secret",
    model_a_name="wine-model-a",
    model_a_version=model_a_version,
    model_b_name="wine-model-b",
    model_b_version=model_b_version,
    model_a_traffic=50,
)

In [ ]:
from pprint import pprint as pp
pp(ab)